In [1]:
from qubolite import qubo
import numpy as np

In [2]:
import sys
sys.path.insert(1, '/Users/linusrandud/Documents/UoM/ERP/MscDissertation/Deep-Opt')

In [3]:
import torch

from COProblems.MKP import MKP
from COProblems.QUBO import QUBO
from Models.DOAE import DOAE
from OptimAE import OptimAEHandler

In [4]:
# Highly recommended to keep as cpu for problems of size <= 100
device="cpu"
print(device)
device = torch.device(device)

cpu


In [5]:
problem = QUBO("/Users/linusrandud/Documents/UoM/ERP/MscDissertation/data/qubo/ssjsp_50.txt", 0, device)

Instance has been loaded


In [6]:
# original

# a = [13, 23, 16, 20]
# solution should be [0, 0, 1, 1] or [1, 1, 0, 0]

In [7]:
problem.Q.detach().cpu().numpy()

array([[-18101.,    253.,    253., ...,    575.,    575.,    345.],
       [   253.,  -8789.,    121., ...,    275.,    275.,    165.],
       [   253.,    121.,  -8789., ...,    275.,    275.,    165.],
       ...,
       [   575.,    275.,    275., ..., -19625.,    625.,    375.],
       [   575.,    275.,    275., ...,    625., -19625.,    375.],
       [   345.,    165.,    165., ...,    375.,    375., -11925.]],
      dtype=float32)

In [8]:
np.triu(problem.Q.detach().cpu().numpy())

array([[-18101.,    253.,    253., ...,    575.,    575.,    345.],
       [     0.,  -8789.,    121., ...,    275.,    275.,    165.],
       [     0.,      0.,  -8789., ...,    275.,    275.,    165.],
       ...,
       [     0.,      0.,      0., ..., -19625.,    625.,    375.],
       [     0.,      0.,      0., ...,      0., -19625.,    375.],
       [     0.,      0.,      0., ...,      0.,      0., -11925.]],
      dtype=float32)

In [9]:
Q = problem.Q.detach().cpu().numpy()

In [10]:
from qubolite.solving import brute_force, local2_descent, local2_descent_search, simulated_annealing

In [11]:
Q_ = qubo(Q)
# brute_force(Q_)

In [12]:
local2_descent(Q_)

qubo_solution(x=array([ True,  True, False, False,  True, False, False,  True,  True,
       False, False,  True,  True,  True,  True, False, False,  True,
        True, False, False,  True,  True, False,  True, False, False,
        True,  True, False,  True,  True, False, False,  True, False,
        True,  True,  True, False, False,  True, False, False,  True,
       False, False, False,  True, False]), energy=-164025.0)

In [13]:
local2_descent_search(Q_)

qubo_solution(x=array([1., 0., 0., 0., 1., 1., 0., 0., 0., 1., 0., 0., 0., 1., 1., 1., 0.,
       0., 0., 1., 0., 0., 0., 0., 0., 0., 1., 0., 0., 1., 1., 0., 0., 1.,
       0., 1., 1., 1., 1., 1., 0., 1., 1., 1., 0., 1., 0., 0., 0., 0.]), energy=-164025.0)

In [14]:
simulated_annealing(Q_)

Init. temp. automatically set to 155851.1513


(array([[1., 0., 0., 0., 1., 0., 1., 0., 1., 0., 1., 1., 0., 1., 1., 0.,
         0., 1., 0., 0., 1., 0., 1., 1., 0., 0., 1., 0., 0., 1., 1., 1.,
         1., 1., 0., 0., 0., 1., 1., 1., 1., 0., 0., 0., 0., 1., 0., 0.,
         1., 1.],
        [0., 1., 1., 1., 0., 1., 1., 1., 0., 1., 1., 0., 0., 1., 1., 0.,
         0., 0., 1., 1., 1., 1., 1., 0., 1., 0., 1., 1., 1., 0., 1., 1.,
         0., 0., 1., 0., 0., 0., 0., 1., 0., 0., 0., 1., 0., 0., 1., 1.,
         0., 1.],
        [1., 1., 0., 1., 0., 1., 0., 0., 1., 1., 0., 1., 0., 0., 1., 1.,
         0., 0., 1., 1., 1., 1., 0., 1., 1., 0., 1., 1., 0., 0., 0., 0.,
         0., 0., 0., 0., 1., 0., 1., 1., 1., 0., 1., 1., 0., 1., 1., 0.,
         0., 0.],
        [0., 0., 0., 1., 1., 0., 0., 0., 0., 1., 1., 1., 0., 0., 0., 1.,
         1., 1., 1., 1., 0., 1., 1., 1., 1., 0., 0., 1., 1., 0., 0., 0.,
         1., 0., 0., 0., 1., 0., 1., 0., 1., 0., 0., 1., 0., 1., 1., 1.,
         1., 0.],
        [1., 1., 0., 1., 0., 1., 1., 1., 0., 1., 0.,

In [88]:
def evaluate_qubo(Q):
    n = Q.shape[0]
    min_value = float('inf')
    best_x = None
    
    for i in range(2**n):
        x = np.array(list(map(int, bin(i)[2:].zfill(n))))
        value = x.T @ Q @ x
        if value < min_value:
            min_value = value
            best_x = x
    
    return best_x, min_value